## Using coordinate batches for training signals in `alpine`

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os
import torch
import skimage.io
import skimage.transform
import skimage.filters
import skimage.data

import alpine
from matplotlib import pyplot as plt

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '3,4'

In [ ]:
H, W = 512, 512
vectorized = True
bounds = (-1, 1)

batch_size = 100 * 100


In [ ]:
signal = skimage.transform.resize(skimage.data.astronaut(), (H, W), anti_aliasing=True)
print(signal.shape)

In [ ]:
coordinate_signal_loader = alpine.dataloaders.NDSignalLoader(
    signal = signal,
    grid_dims = [H, W],
    bounds = bounds,
    normalize_signal = True,
    vectorized = vectorized,
)

In [ ]:
dataloader = torch.utils.data.DataLoader(coordinate_signal_loader, batch_size=1, shuffle=False)

In [ ]:
ffn_model = alpine.models.FFN(
    in_features=2,
    out_features=3,
    hidden_features=256,
    hidden_layers=4,
    positional_encoding='fourier'
).float().cuda()

In [ ]:
for l in ffn_model.model:
    print(list(l.parameters()))

In [ ]:
ret_fit = ffn_model.fit_signal(
    dataloader=dataloader,
    n_iters = 2000,
    track_loss_history = True)

In [ ]:
plt.figure()
plt.imshow(ret_fit['output'].detach().cpu().numpy().reshape(H, W, 3))
plt.show()